<a href="https://colab.research.google.com/github/justinfmccarty/CityEnergyAnalyst_bigmacc/blob/master/Strategy_Control_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import numpy as np
import pandas as pd
import re
import os

In [78]:
def change_key(key):
  s = list(key)
  s[0] = '0'
  s[4] = '0'
  s[5] = '0'
  s[6] = '0'
  return "".join(s)

def copy_key(key,run_list):
  if key in run_list:
    return 'run'
  elif change_key(key) in run_list:
    return os.path.join(change_key(key),'outputs','data','solar-radiation')
  else:
    return np.nan

def runrad_rule(key,run_list): # SETS FOR HEATING REDUCED 1C (20C TO 19C) AND SETS FOR COOLING RAISED 1C (24C TO 25C)
    if key in run_list:
        return True
    else:
        return False

def SP_rule(key,SP_value): # SET INTEGER FOR HEATING AND COOLING SETPOINT
    keys = [int(d) for d in key]
    if keys[0]==1:
        return SP_value
    else:
        return np.nan

def GR_rule(key,GR_value):  # ALL BUILDINGS GET GREEN ROOFS
    keys = [int(d) for d in key]
    if keys[1]==1:
        return GR_value
    else:
        return np.nan

def DR_rule(key,DR_name): # EXISTING BUILDINGS HAVE DEEP WALL AND WINDOW RETROFIT (WWR REDUCED)
    keys = [int(d) for d in key]
    if keys[2] == 1:
        return DR_name
    else:
        return np.nan


def PH_rule(key,PH_value,PH_GR_value): # NEW BUILD REQ PASSIVE
    keys = [int(d) for d in key]
    if keys[3] == 1:
      return PH_value
    else:
      return np.nan

def PH_rule(key,PH_value): # NEW BUILD REQ PASSIVE
    keys = [int(d) for d in key]
    if keys[3] == 1:
      return PH_value
    else:
      return np.nan

def PH_GR_rule(key,PH_GR_value): # NEW BUILD REQ PASSIVE AND GREEN ROOF
    keys = [int(d) for d in key]
    if keys[3] == 1:
      if keys[1] == 1:
        return PH_GR_value
      else:
        return np.nan
    else:
      return np.nan

def HP_rule(key,HP_value):  # NEW BUILD AND REFURBISHMENT/RETROFITS REQUIRE HEAT PUMP INSTALLATION (NO DISTRICT SERVICES)
    keys = [int(d) for d in key]
    if keys[4] == 1:
      return HP_value
    else:
      return np.nan

def MT_rule(key,MT_value):  # NEW BUILD REQUIRES MASS TIMBER STRUCTURAL SYSTEMS
    keys = [int(d) for d in key]
    if keys[5] == 1:
      return MT_value
    else:
      return np.nan

def SW_rule(key,SW_value):  # DISTRICT COOLING SYSTEM INSTALLED LINKED TO WRECK BEACH
    keys = [int(d) for d in key]
    if keys[6] == 1:
      return SW_value
    else:
      return np.nan

def PV_rule(key,PV_value):  # ALL BUILDINGS HAVE ROOFTOP PV INSTALLED
    keys = [int(d) for d in key]
    if keys[7] == 1:
      return PV_value
    else:
      return np.nan

def PV_GR_rule(key,PV_GR_value):  # ALL BUILDINGS HAVE ROOFTOP PV AND GREEN ROOF INSTALLED
    keys = [int(d) for d in key]
    if keys[7] == 1:
      if keys[1] == 1:
        return PV_GR_value
      else:
        return np.nan
    else:
      return np.nan

def PV_PH_rule(key,PV_PH_value):  # ALL BUILDINGS HAVE ROOFTOP PV AND PASSIVE HOUSE INSTALLED
    keys = [int(d) for d in key]
    if keys[7] == 1:
      if keys[3] == 1:
        return PV_PH_value
      else:
        return np.nan
    else:
      return np.nan

def PV_GR_PH_rule(key,PV_GR_PH_value):  # ALL BUILDINGS HAVE ROOFTOP PV AND GREEN ROOF AND PASSIVE HOUSE INSTALLED
    keys = [int(d) for d in key]
    if keys[7] == 1:
      if keys[1] == 1:
        if keys[3] == 1:
          return PV_GR_PH_value
        else:
          return np.nan
      else:
        return np.nan
    else:
        return np.nan


In [90]:
import itertools

def calc_lst(n):
   #https://stackoverflow.com/questions/14931769/how-to-get-all-combination-of-n-binary-value
  return [list(i) for i in itertools.product([0, 1], repeat=n)]

str_list = []
for key in calc_lst(8):
  result = ''.join(str(i) for i in key) 
  str_list.append(result)

def generate_key_list(n):
    # https://stackoverflow.com/questions/14931769/how-to-get-all-combination-of-n-binary-value
    key_list = []
    elements = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for key in elements:
        result = ''.join(str(i) for i in key)
        key_list.append(result)
    return key_list

In [89]:
def rule_dataframe(key_list, run_rad):
  key_df = pd.DataFrame(data={'keys':key_list})

  key_df['SP_heat'] = key_df.apply(lambda x: SP_rule(x['keys'],-1),axis=1)
  key_df['SP_cool'] = key_df.apply(lambda x: SP_rule(x['keys'],1),axis=1)

  key_df['GR_roof'] = key_df.apply(lambda x: GR_rule(x['keys'],'ROOF_AS16'),axis=1)

  key_df['DR_win'] = key_df.apply(lambda x: DR_rule(x['keys'],'WINDOW_AS6'),axis=1)
  key_df['DR_leak'] = key_df.apply(lambda x: DR_rule(x['keys'],'TIGHTNESS_AS2'),axis=1)
  key_df['DR_wall'] = key_df.apply(lambda x: DR_rule(x['keys'],'WALL_AS17'),axis=1)
  key_df['DR_wwr'] = key_df.apply(lambda x: DR_rule(x['keys'],0.20),axis=1)

  key_df['PH_base'] = key_df.apply(lambda x: PH_rule(x['keys'],'FLOOR_AS11'),axis=1)
  key_df['PH_leak'] = key_df.apply(lambda x: PH_rule(x['keys'],'TIGHTNESS_AS1'),axis=1)
  key_df['PH_win'] = key_df.apply(lambda x: PH_rule(x['keys'],'WINDOW_AS6'),axis=1)
  key_df['PH_roof'] = key_df.apply(lambda x: PH_rule(x['keys'],'ROOF_AS17'),axis=1)
  key_df['PH_GR_roof'] = key_df.apply(lambda x: PH_GR_rule(x['keys'],'ROOF_AS18'),axis=1)
  key_df['PH_wall'] = key_df.apply(lambda x: PH_rule(x['keys'],'WALL_AS18'),axis=1)
  key_df['PH_floor'] = key_df.apply(lambda x: PH_rule(x['keys'],'FLOOR_AS12'),axis=1)
  key_df['PH_shade'] = key_df.apply(lambda x: PH_rule(x['keys'],'SHADING_AS4'),axis=1)
  key_df['PH_wwr'] = key_df.apply(lambda x: PH_rule(x['keys'],0.15),axis=1)
  key_df['PH_part'] = key_df.apply(lambda x: PH_rule(x['keys'],'WALL_AS19'),axis=1)

  key_df['HP_hvac_cs'] = key_df.apply(lambda x: HP_rule(x['keys'],'HVAC_COOLING_AS5'),axis=1)
  key_df['HP_hvac_hs'] = key_df.apply(lambda x: HP_rule(x['keys'],'HVAC_HEATING_AS4'),axis=1)
  key_df['HP_supply_cs'] = key_df.apply(lambda x: HP_rule(x['keys'],'SUPPLY_COOLING_AS1'),axis=1)
  key_df['HP_supply_hs'] = key_df.apply(lambda x: HP_rule(x['keys'],'SUPPLY_HEATING_AS7'),axis=1)
  key_df['HP_supply_dhw'] = key_df.apply(lambda x: HP_rule(x['keys'],'SUPPLY_HOTWATER_AS7'),axis=1)

  key_df['MT_cons'] = key_df.apply(lambda x: MT_rule(x['keys'],'CONSTRUCTION_AS2'),axis=1)

  key_df['SW_hvac_cs'] = key_df.apply(lambda x: SW_rule(x['keys'],'HVAC_COOLING_AS5'),axis=1)
  key_df['SW_supply_cs'] = key_df.apply(lambda x: SW_rule(x['keys'],'SUPPLY_COOLING_AS3'),axis=1)

  key_df['PV_GR_PH'] = key_df.apply(lambda x: PV_GR_PH_rule(x['keys'],'ROOF_AS19'),axis=1)
  key_df['PV_PH'] = key_df.apply(lambda x: PV_PH_rule(x['keys'],'ROOF_AS21'),axis=1)
  key_df['PV_GR'] = key_df.apply(lambda x: PV_GR_rule(x['keys'],'ROOF_AS20'),axis=1)
  key_df['PV'] = key_df.apply(lambda x: PV_rule(x['keys'],'ROOF_AS22'),axis=1)

  key_df['PV_GR_PH_hg'] = key_df.apply(lambda x: PV_GR_PH_rule(x['keys'], 0.6), axis=1)
  key_df['PV_PH_hg'] = key_df.apply(lambda x: PV_PH_rule(x['keys'], 0.25), axis=1)
  key_df['PV_GR_hg'] = key_df.apply(lambda x: PV_GR_rule(x['keys'], 0.4), axis=1)
  key_df['PV_hg'] = key_df.apply(lambda x: PV_rule(x['keys'], 0.1), axis=1)
  
  key_df['run_rad'] = key_df.apply(lambda x: runrad_rule(x['keys'],run_rad),axis=1)
  key_df['copy_rad'] = key_df.apply(lambda x: copy_key(x['keys'],run_rad),axis=1)
  return key_df

In [ ]:
def order_key_list(run_list,key_list):
  no_run_rad_list = list(set(key_list) - set(run_list))
  final_list = list()
  final_list.extend(run_list)
  final_list.extend(no_run_rad_list)
  return final_list

run_rad = ['00000000',
            '01000000',
            '01100000',
            '01110000',
            '01110001',
            '00100000',
            '00110000',
            '00100001',
            '00010000',
            '01010000',
            '00010001',
            '00000001',
            '01000001',
            '01100001',
            '01010001',
            '00110001']

run_rad_ids = ['a',
                'b',
                'c',
                'd',
                'e',
                'f',
                'g',
                'h',
                'i',
                'j',
                'k',
                'l',
                'm',
                'n',
                'o',
                'p']

unique_rads = dict(zip(run_rad,run_rad_ids))

order_key_list(run_rad,str_list)

In [92]:
key_df = rule_dataframe(generate_key_list(8), run_rad)

In [93]:
key_df.columns

Index(['keys', 'SP_heat', 'SP_cool', 'GR_roof', 'DR_win', 'DR_leak', 'DR_wall',
       'DR_wwr', 'PH_base', 'PH_leak', 'PH_win', 'PH_roof', 'PH_GR_roof',
       'PH_wall', 'PH_floor', 'PH_shade', 'PH_wwr', 'PH_part', 'HP_hvac_cs',
       'HP_hvac_hs', 'HP_supply_cs', 'HP_supply_hs', 'HP_supply_dhw',
       'MT_cons', 'SW_hvac_cs', 'SW_supply_cs', 'PV_GR_PH', 'PV_PH', 'PV_GR',
       'PV', 'PV_GR_PH_hg', 'PV_PH_hg', 'PV_GR_hg', 'PV_hg', 'run_rad',
       'copy_rad'],
      dtype='object')